In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calc_mutual_inductance(coil_1, coil_2, d, po=0, fi=0):
    mu0 = 4 * np.pi * 10 ** (-7)
    mutual_inductance = np.ones((len(coil_1), len(coil_2)))
    N = 90
    K = 90
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    for ri in range(len(coil_1)):
        for rj in range(len(coil_2)):
            M = 0
            for n in range(N):
                for k in range(K):
                    xk_xn = po + coil_1[ri] * np.cos(df2 * k) * np.cos(fi) - coil_2[rj] * np.cos(df1 * n)
                    yk_yn = coil_1[ri] * np.sin(df2 * k) * np.cos(fi) - coil_2[rj] * np.sin(df1 * n)
                    zk_zn = d + coil_1[ri] * np.cos(df2 * k) * np.sin(fi)
                    r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                    M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
            M *= mu0 * coil_1[ri] * coil_2[rj] / (4 * np.pi)
            mutual_inductance[ri][rj] = M
    return np.sum(mutual_inductance)

def calc_self_inductance(coil, thin=0.001):
    mu0 = 4 * np.pi * 10 ** (-7)
    L = np.sum(mu0 * coil * (np.log(8 * coil / thin) - 7 / 4 + (thin ** 2) / (8 * coil ** 2) * (np.log(8 * coil / thin) + 1 / 3)))
    mutual_inductance = np.ones((len(coil), len(coil)))
    N = 90
    K = 90
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    d = 0
    po = 0
    fi = 0
    for ri in range(len(coil)):
        for rj in range(len(coil)):
            M = 0
            if ri != rj:
                for n in range(N):
                    for k in range(K):
                        xk_xn = po + coil[ri] * np.cos(df2 * k) * np.cos(fi) - coil[rj] * np.cos(df1 * n)
                        yk_yn = coil[ri] * np.sin(df2 * k) * np.cos(fi) - coil[rj] * np.sin(df1 * n)
                        zk_zn = d + coil[ri] * np.cos(df2 * k) * np.sin(fi)
                        r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                        M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
                M *= mu0 * coil[ri] * coil[rj] / (4 * np.pi)
            mutual_inductance[ri][rj] = M
    M = np.sum(mutual_inductance)
    L += M
    return L

def calc_coupling_coefficient(coil_1, coil_2, d, po=0, fi=0):
    M = calc_mutual_inductance(coil_1, coil_2, d, po=po, fi=fi)
    L1 = calc_self_inductance(coil_1)
    L2 = calc_self_inductance(coil_2)
    k = M / (L1 * L2) ** 0.5
    return k

def mutation(start, finish, x, r):
    x = np.random.uniform(low = start if x - r < start else x - r,
                          high = finish if x + r > finish else x + r)
    return x

# между витками одной катушки не должно быть пересечений
def test_coils(coil, thin):
    # между витками 2 и 3 катушки есть пересечение
    if coil[1] > coil[2]:
        if (coil[1] - coil[2] - 2 * thin) < 0:
            return False
    elif coil[2] > coil[1]:
        if (coil[2] - coil[1] - 2 * thin) < 0:
            return False
    return True

In [3]:
def random_mutation_hill_climbing(coil_1, thin, d):
    good_mutation = []
    bad_mutation = []
    all_mutation = []
    
    Q = 2
    band = 0.005
    thr_k = 1e-4
    
    
    start = 0.03 + 2 * thin
    finish = 0.09 - 2 * thin
    coil_2 = np.array([0.03, start, finish, 0.09])
    
    print("Начальное значение катушки:", coil_2)
    i = 0
    while True:
        i += 1
        fit_k = calc_coupling_coefficient(coil_1, coil_2, d)
        # случайно выбираем индекс, который будем мутировать
        q = np.random.randint(0, Q)
        coil_2q = coil_2.copy()
        coil_2q[1 + q] = mutation(start, finish, coil_2q[1 + q], band)
        # проверка на пересечение витков внутри катушки
        while not test_coils(coil_2q, thin):
            print(f"Есть пересечения внутри катушки! {coil_2q}")
            coil_2q[1 + q] = mutation(start, finish, coil_2q[1 + q], band)
        fit_kq = calc_coupling_coefficient(coil_1, coil_2q, d)
        if fit_kq > fit_k:
            coil_2 = coil_2q.copy()
            fit_k = fit_kq
            good_mutation.append((fit_kq, coil_2q))
        else:
            bad_mutation.append((fit_kq, coil_2q))
        all_mutation.append((fit_kq, coil_2q))
        print(f"iter: {i} k = {fit_kq}, катушка = {coil_2q}")
        if len(good_mutation) >= 2:
            if (good_mutation[-1][0] - good_mutation[-2][0]) / thr_k < 1:
                break
        if i == 100:
            print("Алгоритм сделал 100 итераций! Останавливаю алгоритм")
            break
    return good_mutation[-1][0], good_mutation[-1][1], len(good_mutation), len(bad_mutation), len(all_mutation)

    

In [4]:
info_rmhc = []

coil_1 = np.linspace(0.028, 0.07, 4)
thin = 0.001
d = 0.005
N = 10

In [5]:
for i in range(N):
    print(f"{i} итерация запуска алгоритма random mutation hill climbing:", end='\n\n')
    info_rmhc.append(random_mutation_hill_climbing(coil_1, thin, d))

0 итерация запуска алгоритма random mutation hill climbing:

Начальное значение катушки: [0.03  0.032 0.088 0.09 ]
iter: 1 k = 0.4747990037082349, катушка = [0.03       0.032      0.08460853 0.09      ]
iter: 2 k = 0.48446718431246116, катушка = [0.03       0.0338505  0.08460853 0.09      ]
iter: 3 k = 0.4722017062224553, катушка = [0.03       0.0338505  0.08595657 0.09      ]
iter: 4 k = 0.46404167188436624, катушка = [0.03       0.0338505  0.08673318 0.09      ]
iter: 5 k = 0.47464821032261856, катушка = [0.03       0.0338505  0.08570261 0.09      ]
iter: 6 k = 0.5182536912860709, катушка = [0.03      0.0338505 0.0804342 0.09     ]
iter: 7 k = 0.5132593747652944, катушка = [0.03       0.03288096 0.0804342  0.09      ]
iter: 8 k = 0.5292240206882924, катушка = [0.03       0.03611544 0.0804342  0.09      ]
iter: 9 k = 0.5348468863488396, катушка = [0.03       0.03726068 0.0804342  0.09      ]
iter: 10 k = 0.5469085666746398, катушка = [0.03       0.03972146 0.0804342  0.09      ]
iter:

iter: 10 k = 0.519065283156239, катушка = [0.03       0.04172659 0.08480219 0.09      ]
iter: 11 k = 0.5557486888305289, катушка = [0.03       0.04172659 0.08034888 0.09      ]
iter: 12 k = 0.5473815566858347, катушка = [0.03       0.04172659 0.08139127 0.09      ]
iter: 13 k = 0.567729106884022, катушка = [0.03       0.04172659 0.07886813 0.09      ]
iter: 14 k = 0.5570776620067751, катушка = [0.03       0.03918609 0.07886813 0.09      ]
iter: 15 k = 0.5526482772061111, катушка = [0.03       0.03828439 0.07886813 0.09      ]
iter: 16 k = 0.5593485304025606, катушка = [0.03       0.0396637  0.07886813 0.09      ]
iter: 17 k = 0.5741461944521202, катушка = [0.03       0.04172659 0.07808631 0.09      ]
iter: 18 k = 0.5432059919058522, катушка = [0.03       0.04172659 0.08191067 0.09      ]
iter: 19 k = 0.5804798711838417, катушка = [0.03       0.04487006 0.07808631 0.09      ]
iter: 20 k = 0.5775507401435107, катушка = [0.03       0.04299232 0.07808631 0.09      ]
iter: 21 k = 0.58131349

iter: 29 k = 0.5309857031291806, катушка = [0.03       0.04687546 0.08455017 0.09      ]
iter: 30 k = 0.5701817974226738, катушка = [0.03       0.04671457 0.07963038 0.09      ]
iter: 31 k = 0.5680963428052098, катушка = [0.03       0.04490707 0.07963038 0.09      ]
iter: 32 k = 0.5979060334700936, катушка = [0.03       0.04687546 0.0762136  0.09      ]
iter: 33 k = 0.6290584274628558, катушка = [0.03       0.04687546 0.07233342 0.09      ]
iter: 34 k = 0.6292925008638356, катушка = [0.03       0.04715981 0.07233342 0.09      ]
iter: 35 k = 0.6313870700439906, катушка = [0.03       0.04715981 0.07203265 0.09      ]
iter: 36 k = 0.6381217797062195, катушка = [0.03       0.05177904 0.07203265 0.09      ]
iter: 37 k = 0.643238408082555, катушка = [0.03       0.05538917 0.07203265 0.09      ]
iter: 38 k = 0.6528681692634627, катушка = [0.03       0.05538917 0.06989095 0.09      ]
iter: 39 k = 0.6543926103039871, катушка = [0.03       0.05538917 0.0669479  0.09      ]
iter: 40 k = 0.6543228

iter: 47 k = 0.6546001376318703, катушка = [0.03       0.054023   0.06835934 0.09      ]
iter: 48 k = 0.6531895995175564, катушка = [0.03       0.05282564 0.0679862  0.09      ]
iter: 49 k = 0.6466119014649495, катушка = [0.03       0.05873163 0.0679862  0.09      ]
iter: 50 k = 0.6538051713132086, катушка = [0.03      0.054023  0.0691558 0.09     ]
iter: 51 k = 0.6544857390042651, катушка = [0.03       0.054023   0.06730588 0.09      ]
iter: 52 k = 0.6500710912592296, катушка = [0.03       0.054023   0.06309621 0.09      ]
iter: 53 k = 0.6491505251746192, катушка = [0.03       0.05021954 0.0679862  0.09      ]
iter: 54 k = 0.6407301980033187, катушка = [0.03       0.054023   0.07226362 0.09      ]
iter: 55 k = 0.6431047416164142, катушка = [0.03      0.054023  0.0718845 0.09     ]
iter: 56 k = 0.6549626664998538, катушка = [0.03       0.05439083 0.0679862  0.09      ]
iter: 57 k = 0.6545970030392769, катушка = [0.03       0.05592333 0.0679862  0.09      ]
iter: 58 k = 0.65459760141581

iter: 58 k = 0.6468260934021668, катушка = [0.03       0.05870671 0.06813759 0.09      ]
iter: 59 k = 0.6475268196910766, катушка = [0.03       0.05579327 0.07123941 0.09      ]
iter: 60 k = 0.6497282831243907, катушка = [0.03       0.05794635 0.06813759 0.09      ]
iter: 61 k = 0.6454890596972817, катушка = [0.03       0.05902773 0.06813759 0.09      ]
iter: 62 k = 0.6505988745757644, катушка = [0.03       0.05579327 0.06455769 0.09      ]
iter: 63 k = 0.650864456856515, катушка = [0.03       0.05140098 0.06813759 0.09      ]
iter: 64 k = 0.6548601994729415, катушка = [0.03       0.05425201 0.06813759 0.09      ]
6 итерация запуска алгоритма random mutation hill climbing:

Начальное значение катушки: [0.03  0.032 0.088 0.09 ]
iter: 1 k = 0.46242521855414664, катушка = [0.03       0.032      0.08599199 0.09      ]
iter: 2 k = 0.46627694074287585, катушка = [0.03       0.03270151 0.08599199 0.09      ]
iter: 3 k = 0.48490961856772524, катушка = [0.03       0.03270151 0.083872   0.09    

iter: 85 k = 0.6550855795905641, катушка = [0.03       0.05464891 0.06808958 0.09      ]
iter: 86 k = 0.6529533378809445, катушка = [0.03       0.05268567 0.06808958 0.09      ]
iter: 87 k = 0.6515295404801027, катушка = [0.03       0.0573883  0.06808958 0.09      ]
iter: 88 k = 0.6477677500238587, катушка = [0.03       0.05469705 0.07116788 0.09      ]
iter: 89 k = 0.6429381774056244, катушка = [0.03       0.05960109 0.06808958 0.09      ]
iter: 90 k = 0.6546050538477944, катушка = [0.03       0.05469705 0.0669993  0.09      ]
iter: 91 k = 0.6425063320213337, катушка = [0.03       0.05469705 0.07210196 0.09      ]
iter: 92 k = 0.6499405695764587, катушка = [0.03       0.05079629 0.06808958 0.09      ]
iter: 93 k = 0.6538717567466457, катушка = [0.03       0.05469705 0.0662826  0.09      ]
iter: 94 k = 0.6524549568811661, катушка = [0.03       0.05469705 0.0651566  0.09      ]
iter: 95 k = 0.6522933650513363, катушка = [0.03       0.05711939 0.06808958 0.09      ]
iter: 96 k = 0.654995

iter: 21 k = 0.639889030235739, катушка = [0.03       0.04581293 0.07037906 0.09      ]
iter: 22 k = 0.6464122683286339, катушка = [0.03       0.04757205 0.06620678 0.09      ]
iter: 23 k = 0.6457721546837772, катушка = [0.03       0.04757205 0.06192348 0.09      ]
iter: 24 k = 0.6516065625149667, катушка = [0.03       0.05230721 0.06620678 0.09      ]
iter: 25 k = 0.6501585703568454, катушка = [0.03       0.05230721 0.06966804 0.09      ]
iter: 26 k = 0.6520019823725138, катушка = [0.03       0.05230721 0.06863578 0.09      ]
iter: 27 k = 0.6464045963514102, катушка = [0.03       0.0482741  0.06863578 0.09      ]
iter: 28 k = 0.6464793188512745, катушка = [0.03       0.04835846 0.06863578 0.09      ]
iter: 29 k = 0.6520883699383304, катушка = [0.03       0.05719954 0.06863578 0.09      ]


In [6]:
sorted(info_rmhc, key=lambda x: x[0], reverse=True)

[(0.6551397100357101,
  array([0.03      , 0.05489411, 0.06799076, 0.09      ]),
  19,
  62,
  81),
 (0.655117260853698,
  array([0.03      , 0.05485821, 0.06786617, 0.09      ]),
  24,
  48,
  72),
 (0.6551018570462019,
  array([0.03      , 0.05469705, 0.06808958, 0.09      ]),
  21,
  79,
  100),
 (0.6550910875755436,
  array([0.03      , 0.05529084, 0.06821086, 0.09      ]),
  20,
  11,
  31),
 (0.6550818520731176,
  array([0.03      , 0.05467285, 0.06788725, 0.09      ]),
  24,
  50,
  74),
 (0.6549958410390823,
  array([0.03      , 0.05458645, 0.06767155, 0.09      ]),
  21,
  12,
  33),
 (0.6548670293377868,
  array([0.03      , 0.05435865, 0.06761432, 0.09      ]),
  23,
  26,
  49),
 (0.6548601994729415,
  array([0.03      , 0.05425201, 0.06813759, 0.09      ]),
  21,
  43,
  64),
 (0.6520883699383304,
  array([0.03      , 0.05719954, 0.06863578, 0.09      ]),
  19,
  10,
  29),
 (0.6425592396909819,
  array([0.03      , 0.0425059 , 0.06538885, 0.09      ]),
  11,
  9,
  20)]

In [7]:
print(f"Лучший коэффициент связи {info_rmhc[0][0]} у катушки {info_rmhc[0][1]}")
print(f"Среднее количество итераций {sum([i[4] for i in info_rmhc]) / len(info_rmhc)}")
print(f"Среднее количество полезных мутаций {sum([i[2] for i in info_rmhc]) / len(info_rmhc)}")
print(f"Среднее количество бесполезных мутаций {sum([i[3] for i in info_rmhc]) / len(info_rmhc)}")

Лучший коэффициент связи 0.6550910875755436 у катушки [0.03       0.05529084 0.06821086 0.09      ]
Среднее количество итераций 55.3
Среднее количество полезных мутаций 20.3
Среднее количество бесполезных мутаций 35.0
